In [15]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import models
from tensorflow.keras import layers
import pickle


def log(msg):
    print(msg, '\n')


D = dict(pickle.load(open('Oxford-IIIT-Pet_Dics.p', 'rb')))
imageNum = 7390

DKeys = list(D.keys())
log(DKeys)

classDic = dict(D.get('classDic'))
log(classDic)

filesDic = dict(D.get('filesDic'))
filesDicValues = list(filesDic.values())
log(filesDicValues[0])

# log(D)


['classDic', 'filesDic'] 

{'american_bulldog': 0, 'american_pit_bull_terrier': 1, 'basset_hound': 2, 'beagle': 3, 'boxer': 4, 'chihuahua': 5, 'english_cocker_spaniel': 6, 'english_setter': 7, 'german_shorthaired': 8, 'great_pyrenees': 9, 'havanese': 10, 'japanese_chin': 11, 'keeshond': 12, 'leonberger': 13, 'miniature_pinscher': 14, 'newfoundland': 15, 'pomeranian': 16, 'pug': 17, 'saint_bernard': 18, 'samoyed': 19, 'scottish_terrier': 20, 'shiba_inu': 21, 'staffordshire_bull_terrier': 22, 'wheaten_terrier': 23, 'yorkshire_terrier': 24, 'Abyssinian': 25, 'Bengal': 26, 'Birman': 27, 'Bombay': 28, 'British_Shorthair': 29, 'Egyptian_Mau': 30, 'Maine_Coon': 31, 'Persian': 32, 'Ragdoll': 33, 'Russian_Blue': 34, 'Siamese': 35, 'Sphynx': 36} 

{'species': 'cat', 'breed': 'Abyssinian', 'target': 25, 'imgDim': (400, 600, 3), 'bndBox': [333, 72, 92, 86], 'foldTrain': True} 



In [ ]:
cnn = models.Sequential()

cnn.add(layers.Conv2D(16, (5, 5), activation='relu',
                      input_shape=(28, 28, 1), padding="same"))
cnn.add(layers.Conv2D(16, (5, 5), activation='relu'))
cnn.add(layers.AveragePooling2D((2, 2)))
cnn.add(layers.Conv2D(64, (5, 5), activation='relu'))
cnn.add(layers.Conv2D(64, (5, 5), activation='relu'))
cnn.add(layers.AveragePooling2D((2, 2)))
cnn.add(layers.Conv2D(128, (1, 1), activation='relu'))
cnn.add(layers.Conv2D(128, (1, 1), activation='relu'))
cnn.add(layers.AveragePooling2D((2, 2)))
cnn.add(layers.Flatten())
cnn.add(layers.Dense(96, activation='relu'))
cnn.add(layers.Dense(10, activation='softmax'))

cnn.summary()

cnn.compile(optimizer="nadam",
            loss="categorical_crossentropy",
            metrics=["accuracy"])
